In [0]:
!pip install SimpleITK==1.2.4 keras-tqdm==2.0.1 opencv-python==3.4.2.17

     |████████████████████████████████| 42.5MB 101kB/s 
     |████████████████████████████████| 25.0MB 179kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('/content/gdrive/My Drive/3d-mri-brain-tumor-segmentation-using-autoencoder-regularization-master')
print(os.getcwd())

/content/gdrive/My Drive/3d-mri-brain-tumor-segmentation-using-autoencoder-regularization-master


In [0]:
import SimpleITK as sitk  # For loading the dataset
import numpy as np  # For data manipulation
from model import build_model  # For creating the model
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage import zoom
import cv2
import keras.backend as K
from keras.utils import Sequence
from keras.callbacks import LambdaCallback
import cv2

Using TensorFlow backend.


In [0]:
def read_img(img_path):
    """
    Reads a .nii.gz image and returns as a numpy array.
    """
    return sitk.GetArrayFromImage(sitk.ReadImage(img_path))


def resize(img, shape, mode='constant', orig_shape=(155, 240, 240)):
    """
    Wrapper for scipy.ndimage.zoom suited for MRI images.
    """
    assert len(shape) == 3, "Can not have more than 3 dimensions"
    factors = (
        shape[0]/orig_shape[0],
        shape[1]/orig_shape[1], 
        shape[2]/orig_shape[2]
    )
    
    # Resize to the given shape
    return zoom(img, factors, mode=mode)


def preprocess(img, out_shape=None):
    """
    Preprocess the image.
    Just an example, you can add more preprocessing steps if you wish to.
    """
    if out_shape is not None:
        img = resize(img, out_shape, mode='constant')
    
    # Normalize the image
    mean = img.mean()
    std = img.std()
    return (img - mean) / std


def preprocess_label(img, out_shape=None, mode='nearest'):
    """
    Separates out the 3 labels from the segmentation provided, namely:
    GD-enhancing tumor (ET ? label 4), the peritumoral edema (ED ? label 2))
    and the necrotic and non-enhancing tumor core (NCR/NET ? label 1)
    """
    # print(img.shape)
    # print(np.unique(img))
    ncr = img == 1  # Necrotic and Non-Enhancing Tumor (NCR/NET)
    ed = img == 2  # Peritumoral Edema (ED)
    et = img == 4  # GD-enhancing Tumor (ET)
    
    if out_shape is not None:
        ncr = resize(ncr, out_shape, mode=mode)
        ed = resize(ed, out_shape, mode=mode)
        et = resize(et, out_shape, mode=mode)

    return np.array([ncr, ed, et], dtype=np.uint8)

def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
    return (2. * intersection) / (
        K.sum(K.square(y_true_f), -1) + K.sum(K.square(y_pred_f), -1) + 1e-8)

In [0]:
input_shape = (4, 96, 112, 112)
output_channels = 3
model = build_model(input_shape=input_shape, output_channels=3)
model.load_weights('weights/weights.epoch_100-loss_-0.14610-dice_0.64230-val_dice_0.58464.hdf5')

#train samples
samples=[{'t1': 'LGG/Brats18_TCIA10_152_1/Brats18_TCIA10_152_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_152_1/Brats18_TCIA10_152_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_152_1/Brats18_TCIA10_152_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_152_1/Brats18_TCIA10_152_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_152_1/Brats18_TCIA10_152_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_0_1/Brats18_2013_0_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_0_1/Brats18_2013_0_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_0_1/Brats18_2013_0_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_0_1/Brats18_2013_0_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_0_1/Brats18_2013_0_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_634_1/Brats18_TCIA13_634_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_634_1/Brats18_TCIA13_634_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_634_1/Brats18_TCIA13_634_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_634_1/Brats18_TCIA13_634_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_634_1/Brats18_TCIA13_634_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_330_1/Brats18_TCIA10_330_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_330_1/Brats18_TCIA10_330_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_330_1/Brats18_TCIA10_330_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_330_1/Brats18_TCIA10_330_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_330_1/Brats18_TCIA10_330_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_637_1/Brats18_TCIA10_637_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_637_1/Brats18_TCIA10_637_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_637_1/Brats18_TCIA10_637_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_637_1/Brats18_TCIA10_637_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_637_1/Brats18_TCIA10_637_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_261_1/Brats18_TCIA10_261_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_261_1/Brats18_TCIA10_261_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_261_1/Brats18_TCIA10_261_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_261_1/Brats18_TCIA10_261_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_261_1/Brats18_TCIA10_261_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_632_1/Brats18_TCIA10_632_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_632_1/Brats18_TCIA10_632_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_632_1/Brats18_TCIA10_632_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_632_1/Brats18_TCIA10_632_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_632_1/Brats18_TCIA10_632_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA12_101_1/Brats18_TCIA12_101_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA12_101_1/Brats18_TCIA12_101_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA12_101_1/Brats18_TCIA12_101_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA12_101_1/Brats18_TCIA12_101_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA12_101_1/Brats18_TCIA12_101_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_266_1/Brats18_TCIA10_266_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_15_1/Brats18_2013_15_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_15_1/Brats18_2013_15_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_15_1/Brats18_2013_15_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_15_1/Brats18_2013_15_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_15_1/Brats18_2013_15_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_653_1/Brats18_TCIA13_653_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_653_1/Brats18_TCIA13_653_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_653_1/Brats18_TCIA13_653_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_653_1/Brats18_TCIA13_653_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_653_1/Brats18_TCIA13_653_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA12_298_1/Brats18_TCIA12_298_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA12_298_1/Brats18_TCIA12_298_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA12_298_1/Brats18_TCIA12_298_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA12_298_1/Brats18_TCIA12_298_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA12_298_1/Brats18_TCIA12_298_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_16_1/Brats18_2013_16_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_16_1/Brats18_2013_16_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_16_1/Brats18_2013_16_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_16_1/Brats18_2013_16_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_16_1/Brats18_2013_16_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_312_1/Brats18_TCIA09_312_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_312_1/Brats18_TCIA09_312_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_312_1/Brats18_TCIA09_312_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_312_1/Brats18_TCIA09_312_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_312_1/Brats18_TCIA09_312_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_24_1/Brats18_2013_24_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_24_1/Brats18_2013_24_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_24_1/Brats18_2013_24_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_24_1/Brats18_2013_24_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_24_1/Brats18_2013_24_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_449_1/Brats18_TCIA10_449_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_449_1/Brats18_TCIA10_449_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_449_1/Brats18_TCIA10_449_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_449_1/Brats18_TCIA10_449_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_449_1/Brats18_TCIA10_449_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_628_1/Brats18_TCIA10_628_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_628_1/Brats18_TCIA10_628_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_628_1/Brats18_TCIA10_628_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_628_1/Brats18_TCIA10_628_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_628_1/Brats18_TCIA10_628_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_254_1/Brats18_TCIA09_254_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_254_1/Brats18_TCIA09_254_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_254_1/Brats18_TCIA09_254_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_254_1/Brats18_TCIA09_254_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_254_1/Brats18_TCIA09_254_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_8_1/Brats18_2013_8_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_8_1/Brats18_2013_8_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_8_1/Brats18_2013_8_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_8_1/Brats18_2013_8_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_8_1/Brats18_2013_8_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_29_1/Brats18_2013_29_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_29_1/Brats18_2013_29_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_29_1/Brats18_2013_29_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_29_1/Brats18_2013_29_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_29_1/Brats18_2013_29_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_413_1/Brats18_TCIA10_413_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_413_1/Brats18_TCIA10_413_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_413_1/Brats18_TCIA10_413_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_413_1/Brats18_TCIA10_413_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_413_1/Brats18_TCIA10_413_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_633_1/Brats18_TCIA13_633_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_387_1/Brats18_TCIA10_387_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_387_1/Brats18_TCIA10_387_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_387_1/Brats18_TCIA10_387_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_387_1/Brats18_TCIA10_387_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_387_1/Brats18_TCIA10_387_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA12_480_1/Brats18_TCIA12_480_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA12_480_1/Brats18_TCIA12_480_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA12_480_1/Brats18_TCIA12_480_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA12_480_1/Brats18_TCIA12_480_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA12_480_1/Brats18_TCIA12_480_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_346_1/Brats18_TCIA10_346_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_346_1/Brats18_TCIA10_346_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_346_1/Brats18_TCIA10_346_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_346_1/Brats18_TCIA10_346_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_346_1/Brats18_TCIA10_346_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_325_1/Brats18_TCIA10_325_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_325_1/Brats18_TCIA10_325_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_325_1/Brats18_TCIA10_325_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_325_1/Brats18_TCIA10_325_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_325_1/Brats18_TCIA10_325_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_255_1/Brats18_TCIA09_255_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_255_1/Brats18_TCIA09_255_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_255_1/Brats18_TCIA09_255_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_255_1/Brats18_TCIA09_255_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_255_1/Brats18_TCIA09_255_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_177_1/Brats18_TCIA09_177_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_630_1/Brats18_TCIA13_630_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_630_1/Brats18_TCIA13_630_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_630_1/Brats18_TCIA13_630_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_630_1/Brats18_TCIA13_630_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_630_1/Brats18_TCIA13_630_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_299_1/Brats18_TCIA10_299_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_299_1/Brats18_TCIA10_299_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_299_1/Brats18_TCIA10_299_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_299_1/Brats18_TCIA10_299_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_299_1/Brats18_TCIA10_299_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_624_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_624_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_624_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_624_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_624_1/Brats18_TCIA13_624_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_130_1/Brats18_TCIA10_130_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_130_1/Brats18_TCIA10_130_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_130_1/Brats18_TCIA10_130_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_130_1/Brats18_TCIA10_130_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_130_1/Brats18_TCIA10_130_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_615_1/Brats18_TCIA13_615_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_615_1/Brats18_TCIA13_615_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_615_1/Brats18_TCIA13_615_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_615_1/Brats18_TCIA13_615_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_615_1/Brats18_TCIA13_615_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA12_249_1/Brats18_TCIA12_249_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA12_249_1/Brats18_TCIA12_249_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA12_249_1/Brats18_TCIA12_249_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA12_249_1/Brats18_TCIA12_249_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA12_249_1/Brats18_TCIA12_249_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_241_1/Brats18_TCIA10_241_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_241_1/Brats18_TCIA10_241_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_241_1/Brats18_TCIA10_241_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_241_1/Brats18_TCIA10_241_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_241_1/Brats18_TCIA10_241_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_642_1/Brats18_TCIA13_642_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_618_1/Brats18_TCIA13_618_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_618_1/Brats18_TCIA13_618_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_618_1/Brats18_TCIA13_618_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_618_1/Brats18_TCIA13_618_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_618_1/Brats18_TCIA13_618_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_6_1/Brats18_2013_6_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_6_1/Brats18_2013_6_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_6_1/Brats18_2013_6_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_6_1/Brats18_2013_6_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_6_1/Brats18_2013_6_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_645_1/Brats18_TCIA13_645_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA12_466_1/Brats18_TCIA12_466_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA12_466_1/Brats18_TCIA12_466_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA12_466_1/Brats18_TCIA12_466_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA12_466_1/Brats18_TCIA12_466_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA12_466_1/Brats18_TCIA12_466_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_408_1/Brats18_TCIA10_408_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_408_1/Brats18_TCIA10_408_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_408_1/Brats18_TCIA10_408_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_408_1/Brats18_TCIA10_408_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_408_1/Brats18_TCIA10_408_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA12_470_1/Brats18_TCIA12_470_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_428_1/Brats18_TCIA09_428_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_428_1/Brats18_TCIA09_428_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_428_1/Brats18_TCIA09_428_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_428_1/Brats18_TCIA09_428_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_428_1/Brats18_TCIA09_428_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_410_1/Brats18_TCIA10_410_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_410_1/Brats18_TCIA10_410_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_410_1/Brats18_TCIA10_410_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_410_1/Brats18_TCIA10_410_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_410_1/Brats18_TCIA10_410_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_351_1/Brats18_TCIA10_351_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_351_1/Brats18_TCIA10_351_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_351_1/Brats18_TCIA10_351_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_351_1/Brats18_TCIA10_351_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_351_1/Brats18_TCIA10_351_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_629_1/Brats18_TCIA10_629_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_28_1/Brats18_2013_28_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_28_1/Brats18_2013_28_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_28_1/Brats18_2013_28_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_28_1/Brats18_2013_28_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_28_1/Brats18_2013_28_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_451_1/Brats18_TCIA09_451_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_451_1/Brats18_TCIA09_451_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_451_1/Brats18_TCIA09_451_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_451_1/Brats18_TCIA09_451_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_451_1/Brats18_TCIA09_451_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_654_1/Brats18_TCIA13_654_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_654_1/Brats18_TCIA13_654_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_654_1/Brats18_TCIA13_654_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_654_1/Brats18_TCIA13_654_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_654_1/Brats18_TCIA13_654_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_420_1/Brats18_TCIA10_420_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_420_1/Brats18_TCIA10_420_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_420_1/Brats18_TCIA10_420_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_420_1/Brats18_TCIA10_420_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_420_1/Brats18_TCIA10_420_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_282_1/Brats18_TCIA10_282_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_282_1/Brats18_TCIA10_282_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_282_1/Brats18_TCIA10_282_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_282_1/Brats18_TCIA10_282_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_282_1/Brats18_TCIA10_282_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_640_1/Brats18_TCIA10_640_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_640_1/Brats18_TCIA10_640_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_640_1/Brats18_TCIA10_640_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_640_1/Brats18_TCIA10_640_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_640_1/Brats18_TCIA10_640_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_310_1/Brats18_TCIA10_310_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_103_1/Brats18_TCIA10_103_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_103_1/Brats18_TCIA10_103_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_103_1/Brats18_TCIA10_103_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_103_1/Brats18_TCIA10_103_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_103_1/Brats18_TCIA10_103_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_490_1/Brats18_TCIA10_490_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_490_1/Brats18_TCIA10_490_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_490_1/Brats18_TCIA10_490_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_490_1/Brats18_TCIA10_490_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_490_1/Brats18_TCIA10_490_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_175_1/Brats18_TCIA10_175_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_175_1/Brats18_TCIA10_175_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_175_1/Brats18_TCIA10_175_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_175_1/Brats18_TCIA10_175_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_175_1/Brats18_TCIA10_175_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_1_1/Brats18_2013_1_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_1_1/Brats18_2013_1_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_1_1/Brats18_2013_1_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_1_1/Brats18_2013_1_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_1_1/Brats18_2013_1_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_493_1/Brats18_TCIA09_493_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_493_1/Brats18_TCIA09_493_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_493_1/Brats18_TCIA09_493_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_493_1/Brats18_TCIA09_493_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_493_1/Brats18_TCIA09_493_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_623_1/Brats18_TCIA13_623_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_623_1/Brats18_TCIA13_623_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_623_1/Brats18_TCIA13_623_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_623_1/Brats18_TCIA13_623_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_623_1/Brats18_TCIA13_623_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_621_1/Brats18_TCIA13_621_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_621_1/Brats18_TCIA13_621_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_621_1/Brats18_TCIA13_621_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_621_1/Brats18_TCIA13_621_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_621_1/Brats18_TCIA13_621_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_307_1/Brats18_TCIA10_307_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_307_1/Brats18_TCIA10_307_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_307_1/Brats18_TCIA10_307_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_307_1/Brats18_TCIA10_307_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_307_1/Brats18_TCIA10_307_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_625_1/Brats18_TCIA10_625_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_625_1/Brats18_TCIA10_625_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_625_1/Brats18_TCIA10_625_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_625_1/Brats18_TCIA10_625_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_625_1/Brats18_TCIA10_625_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_141_1/Brats18_TCIA09_141_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_276_1/Brats18_TCIA10_276_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_276_1/Brats18_TCIA10_276_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_276_1/Brats18_TCIA10_276_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_276_1/Brats18_TCIA10_276_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_276_1/Brats18_TCIA10_276_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_639_1/Brats18_TCIA10_639_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_639_1/Brats18_TCIA10_639_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_639_1/Brats18_TCIA10_639_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_639_1/Brats18_TCIA10_639_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_639_1/Brats18_TCIA10_639_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_402_1/Brats18_TCIA09_402_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_402_1/Brats18_TCIA09_402_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_402_1/Brats18_TCIA09_402_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_402_1/Brats18_TCIA09_402_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_402_1/Brats18_TCIA09_402_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_644_1/Brats18_TCIA10_644_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_620_1/Brats18_TCIA09_620_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA13_650_1/Brats18_TCIA13_650_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA13_650_1/Brats18_TCIA13_650_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA13_650_1/Brats18_TCIA13_650_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA13_650_1/Brats18_TCIA13_650_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA13_650_1/Brats18_TCIA13_650_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_109_1/Brats18_TCIA10_109_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_109_1/Brats18_TCIA10_109_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_109_1/Brats18_TCIA10_109_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_109_1/Brats18_TCIA10_109_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_109_1/Brats18_TCIA10_109_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_202_1/Brats18_TCIA10_202_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA09_462_1/Brats18_TCIA09_462_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_2013_9_1/Brats18_2013_9_1_t1.nii.gz', 't2': 'LGG/Brats18_2013_9_1/Brats18_2013_9_1_t2.nii.gz', 't1ce': 'LGG/Brats18_2013_9_1/Brats18_2013_9_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_2013_9_1/Brats18_2013_9_1_flair.nii.gz', 'seg': 'LGG/Brats18_2013_9_1/Brats18_2013_9_1_seg.nii.gz'}, {'t1': 'LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_t1.nii.gz', 't2': 'LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_t2.nii.gz', 't1ce': 'LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_t1ce.nii.gz', 'flair': 'LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_flair.nii.gz', 'seg': 'LGG/Brats18_TCIA10_393_1/Brats18_TCIA10_393_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_430_1/Brats18_TCIA02_430_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_430_1/Brats18_TCIA02_430_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_430_1/Brats18_TCIA02_430_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_430_1/Brats18_TCIA02_430_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_430_1/Brats18_TCIA02_430_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_390_1/Brats18_TCIA01_390_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_390_1/Brats18_TCIA01_390_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_390_1/Brats18_TCIA01_390_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_390_1/Brats18_TCIA01_390_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_390_1/Brats18_TCIA01_390_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_479_1/Brats18_TCIA04_479_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_479_1/Brats18_TCIA04_479_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_479_1/Brats18_TCIA04_479_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_479_1/Brats18_TCIA04_479_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_479_1/Brats18_TCIA04_479_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_BHB_1/Brats18_CBICA_BHB_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_BHB_1/Brats18_CBICA_BHB_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_BHB_1/Brats18_CBICA_BHB_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_BHB_1/Brats18_CBICA_BHB_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_BHB_1/Brats18_CBICA_BHB_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AXM_1/Brats18_CBICA_AXM_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AXM_1/Brats18_CBICA_AXM_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AXM_1/Brats18_CBICA_AXM_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AXM_1/Brats18_CBICA_AXM_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AXM_1/Brats18_CBICA_AXM_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_180_1/Brats18_TCIA01_180_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_180_1/Brats18_TCIA01_180_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_180_1/Brats18_TCIA01_180_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_180_1/Brats18_TCIA01_180_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_180_1/Brats18_TCIA01_180_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASO_1/Brats18_CBICA_ASO_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASO_1/Brats18_CBICA_ASO_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASO_1/Brats18_CBICA_ASO_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASO_1/Brats18_CBICA_ASO_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASO_1/Brats18_CBICA_ASO_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_319_1/Brats18_TCIA08_319_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_319_1/Brats18_TCIA08_319_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_319_1/Brats18_TCIA08_319_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_319_1/Brats18_TCIA08_319_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_319_1/Brats18_TCIA08_319_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_25_1/Brats18_2013_25_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_25_1/Brats18_2013_25_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_25_1/Brats18_2013_25_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_25_1/Brats18_2013_25_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_25_1/Brats18_2013_25_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_2_1/Brats18_2013_2_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_2_1/Brats18_2013_2_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_2_1/Brats18_2013_2_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_2_1/Brats18_2013_2_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_2_1/Brats18_2013_2_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_436_1/Brats18_TCIA08_436_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_436_1/Brats18_TCIA08_436_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_436_1/Brats18_TCIA08_436_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_436_1/Brats18_TCIA08_436_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_436_1/Brats18_TCIA08_436_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ABE_1/Brats18_CBICA_ABE_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ABE_1/Brats18_CBICA_ABE_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ABE_1/Brats18_CBICA_ABE_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ABE_1/Brats18_CBICA_ABE_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ABE_1/Brats18_CBICA_ABE_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_203_1/Brats18_TCIA01_203_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_203_1/Brats18_TCIA01_203_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_203_1/Brats18_TCIA01_203_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_203_1/Brats18_TCIA01_203_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_203_1/Brats18_TCIA01_203_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_22_1/Brats18_2013_22_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_22_1/Brats18_2013_22_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_22_1/Brats18_2013_22_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_22_1/Brats18_2013_22_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_22_1/Brats18_2013_22_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_290_1/Brats18_TCIA02_290_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_290_1/Brats18_TCIA02_290_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_290_1/Brats18_TCIA02_290_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_290_1/Brats18_TCIA02_290_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_290_1/Brats18_TCIA02_290_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASK_1/Brats18_CBICA_ASK_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASK_1/Brats18_CBICA_ASK_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASK_1/Brats18_CBICA_ASK_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASK_1/Brats18_CBICA_ASK_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASK_1/Brats18_CBICA_ASK_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_131_1/Brats18_TCIA01_131_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_131_1/Brats18_TCIA01_131_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_131_1/Brats18_TCIA01_131_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_131_1/Brats18_TCIA01_131_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_131_1/Brats18_TCIA01_131_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AOP_1/Brats18_CBICA_AOP_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AOP_1/Brats18_CBICA_AOP_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AOP_1/Brats18_CBICA_AOP_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AOP_1/Brats18_CBICA_AOP_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AOP_1/Brats18_CBICA_AOP_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_13_1/Brats18_2013_13_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_13_1/Brats18_2013_13_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_13_1/Brats18_2013_13_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_13_1/Brats18_2013_13_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_13_1/Brats18_2013_13_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ABO_1/Brats18_CBICA_ABO_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ABO_1/Brats18_CBICA_ABO_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ABO_1/Brats18_CBICA_ABO_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ABO_1/Brats18_CBICA_ABO_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ABO_1/Brats18_CBICA_ABO_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQG_1/Brats18_CBICA_AQG_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQG_1/Brats18_CBICA_AQG_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQG_1/Brats18_CBICA_AQG_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQG_1/Brats18_CBICA_AQG_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQG_1/Brats18_CBICA_AQG_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASY_1/Brats18_CBICA_ASY_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASY_1/Brats18_CBICA_ASY_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASY_1/Brats18_CBICA_ASY_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASY_1/Brats18_CBICA_ASY_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASY_1/Brats18_CBICA_ASY_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AYU_1/Brats18_CBICA_AYU_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AYU_1/Brats18_CBICA_AYU_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AYU_1/Brats18_CBICA_AYU_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AYU_1/Brats18_CBICA_AYU_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AYU_1/Brats18_CBICA_AYU_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AVV_1/Brats18_CBICA_AVV_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AVV_1/Brats18_CBICA_AVV_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AVV_1/Brats18_CBICA_AVV_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AVV_1/Brats18_CBICA_AVV_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AVV_1/Brats18_CBICA_AVV_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_111_1/Brats18_TCIA04_111_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_111_1/Brats18_TCIA04_111_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_111_1/Brats18_TCIA04_111_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_111_1/Brats18_TCIA04_111_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_111_1/Brats18_TCIA04_111_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_192_1/Brats18_TCIA04_192_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_192_1/Brats18_TCIA04_192_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_192_1/Brats18_TCIA04_192_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_192_1/Brats18_TCIA04_192_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_192_1/Brats18_TCIA04_192_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_221_1/Brats18_TCIA01_221_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_221_1/Brats18_TCIA01_221_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_221_1/Brats18_TCIA01_221_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_221_1/Brats18_TCIA01_221_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_221_1/Brats18_TCIA01_221_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AYI_1/Brats18_CBICA_AYI_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AYI_1/Brats18_CBICA_AYI_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AYI_1/Brats18_CBICA_AYI_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AYI_1/Brats18_CBICA_AYI_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AYI_1/Brats18_CBICA_AYI_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_331_1/Brats18_TCIA02_331_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_331_1/Brats18_TCIA02_331_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_331_1/Brats18_TCIA02_331_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_331_1/Brats18_TCIA02_331_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_331_1/Brats18_TCIA02_331_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ATV_1/Brats18_CBICA_ATV_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ATV_1/Brats18_CBICA_ATV_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ATV_1/Brats18_CBICA_ATV_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ATV_1/Brats18_CBICA_ATV_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ATV_1/Brats18_CBICA_ATV_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_179_1/Brats18_TCIA02_179_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_179_1/Brats18_TCIA02_179_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_179_1/Brats18_TCIA02_179_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_179_1/Brats18_TCIA02_179_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_179_1/Brats18_TCIA02_179_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_474_1/Brats18_TCIA03_474_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_474_1/Brats18_TCIA03_474_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_474_1/Brats18_TCIA03_474_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_474_1/Brats18_TCIA03_474_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_474_1/Brats18_TCIA03_474_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_406_1/Brats18_TCIA08_406_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_406_1/Brats18_TCIA08_406_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_406_1/Brats18_TCIA08_406_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_406_1/Brats18_TCIA08_406_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_406_1/Brats18_TCIA08_406_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_437_1/Brats18_TCIA04_437_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_437_1/Brats18_TCIA04_437_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_437_1/Brats18_TCIA04_437_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_437_1/Brats18_TCIA04_437_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_437_1/Brats18_TCIA04_437_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_205_1/Brats18_TCIA08_205_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_205_1/Brats18_TCIA08_205_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_205_1/Brats18_TCIA08_205_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_205_1/Brats18_TCIA08_205_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_205_1/Brats18_TCIA08_205_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_3_1/Brats18_2013_3_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_3_1/Brats18_2013_3_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_3_1/Brats18_2013_3_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_3_1/Brats18_2013_3_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_3_1/Brats18_2013_3_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_117_1/Brats18_TCIA02_117_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_117_1/Brats18_TCIA02_117_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_117_1/Brats18_TCIA02_117_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_117_1/Brats18_TCIA02_117_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_117_1/Brats18_TCIA02_117_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_409_1/Brats18_TCIA06_409_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_409_1/Brats18_TCIA06_409_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_409_1/Brats18_TCIA06_409_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_409_1/Brats18_TCIA06_409_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_409_1/Brats18_TCIA06_409_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA05_396_1/Brats18_TCIA05_396_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA05_396_1/Brats18_TCIA05_396_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA05_396_1/Brats18_TCIA05_396_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA05_396_1/Brats18_TCIA05_396_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA05_396_1/Brats18_TCIA05_396_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AOZ_1/Brats18_CBICA_AOZ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AOZ_1/Brats18_CBICA_AOZ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AOZ_1/Brats18_CBICA_AOZ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AOZ_1/Brats18_CBICA_AOZ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AOZ_1/Brats18_CBICA_AOZ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_208_1/Brats18_TCIA02_208_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_208_1/Brats18_TCIA02_208_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_208_1/Brats18_TCIA02_208_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_208_1/Brats18_TCIA02_208_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_208_1/Brats18_TCIA02_208_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ABM_1/Brats18_CBICA_ABM_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ABM_1/Brats18_CBICA_ABM_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ABM_1/Brats18_CBICA_ABM_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ABM_1/Brats18_CBICA_ABM_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ABM_1/Brats18_CBICA_ABM_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_425_1/Brats18_TCIA01_425_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_425_1/Brats18_TCIA01_425_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_425_1/Brats18_TCIA01_425_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_425_1/Brats18_TCIA01_425_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_425_1/Brats18_TCIA01_425_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_BFP_1/Brats18_CBICA_BFP_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_BFP_1/Brats18_CBICA_BFP_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_BFP_1/Brats18_CBICA_BFP_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_BFP_1/Brats18_CBICA_BFP_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_BFP_1/Brats18_CBICA_BFP_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQT_1/Brats18_CBICA_AQT_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQT_1/Brats18_CBICA_AQT_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQT_1/Brats18_CBICA_AQT_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQT_1/Brats18_CBICA_AQT_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQT_1/Brats18_CBICA_AQT_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_APR_1/Brats18_CBICA_APR_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_APR_1/Brats18_CBICA_APR_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_APR_1/Brats18_CBICA_APR_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_APR_1/Brats18_CBICA_APR_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_APR_1/Brats18_CBICA_APR_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AXN_1/Brats18_CBICA_AXN_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AXN_1/Brats18_CBICA_AXN_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AXN_1/Brats18_CBICA_AXN_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AXN_1/Brats18_CBICA_AXN_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AXN_1/Brats18_CBICA_AXN_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_471_1/Brats18_TCIA02_471_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_471_1/Brats18_TCIA02_471_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_471_1/Brats18_TCIA02_471_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_471_1/Brats18_TCIA02_471_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_471_1/Brats18_TCIA02_471_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_314_1/Brats18_TCIA02_314_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_314_1/Brats18_TCIA02_314_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_314_1/Brats18_TCIA02_314_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_314_1/Brats18_TCIA02_314_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_314_1/Brats18_TCIA02_314_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AAL_1/Brats18_CBICA_AAL_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AAL_1/Brats18_CBICA_AAL_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AAL_1/Brats18_CBICA_AAL_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AAL_1/Brats18_CBICA_AAL_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AAL_1/Brats18_CBICA_AAL_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_309_1/Brats18_TCIA02_309_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_309_1/Brats18_TCIA02_309_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_309_1/Brats18_TCIA02_309_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_309_1/Brats18_TCIA02_309_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_309_1/Brats18_TCIA02_309_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_12_1/Brats18_2013_12_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_12_1/Brats18_2013_12_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_12_1/Brats18_2013_12_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_12_1/Brats18_2013_12_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_12_1/Brats18_2013_12_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AXL_1/Brats18_CBICA_AXL_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AXL_1/Brats18_CBICA_AXL_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AXL_1/Brats18_CBICA_AXL_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AXL_1/Brats18_CBICA_AXL_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AXL_1/Brats18_CBICA_AXL_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ATX_1/Brats18_CBICA_ATX_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ATX_1/Brats18_CBICA_ATX_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ATX_1/Brats18_CBICA_ATX_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ATX_1/Brats18_CBICA_ATX_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ATX_1/Brats18_CBICA_ATX_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_BHM_1/Brats18_CBICA_BHM_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_BHM_1/Brats18_CBICA_BHM_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_BHM_1/Brats18_CBICA_BHM_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_BHM_1/Brats18_CBICA_BHM_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_BHM_1/Brats18_CBICA_BHM_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_190_1/Brats18_TCIA01_190_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_190_1/Brats18_TCIA01_190_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_190_1/Brats18_TCIA01_190_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_190_1/Brats18_TCIA01_190_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_190_1/Brats18_TCIA01_190_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_280_1/Brats18_TCIA08_280_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_280_1/Brats18_TCIA08_280_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_280_1/Brats18_TCIA08_280_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_280_1/Brats18_TCIA08_280_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_280_1/Brats18_TCIA08_280_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AWI_1/Brats18_CBICA_AWI_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AWI_1/Brats18_CBICA_AWI_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AWI_1/Brats18_CBICA_AWI_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AWI_1/Brats18_CBICA_AWI_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AWI_1/Brats18_CBICA_AWI_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_135_1/Brats18_TCIA02_135_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_135_1/Brats18_TCIA02_135_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_135_1/Brats18_TCIA02_135_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_135_1/Brats18_TCIA02_135_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_135_1/Brats18_TCIA02_135_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ARW_1/Brats18_CBICA_ARW_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ARW_1/Brats18_CBICA_ARW_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ARW_1/Brats18_CBICA_ARW_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ARW_1/Brats18_CBICA_ARW_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ARW_1/Brats18_CBICA_ARW_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_186_1/Brats18_TCIA01_186_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_186_1/Brats18_TCIA01_186_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_186_1/Brats18_TCIA01_186_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_186_1/Brats18_TCIA01_186_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_186_1/Brats18_TCIA01_186_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_235_1/Brats18_TCIA01_235_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_235_1/Brats18_TCIA01_235_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_235_1/Brats18_TCIA01_235_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_235_1/Brats18_TCIA01_235_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_235_1/Brats18_TCIA01_235_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_605_1/Brats18_TCIA02_605_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_605_1/Brats18_TCIA02_605_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_605_1/Brats18_TCIA02_605_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_605_1/Brats18_TCIA02_605_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_605_1/Brats18_TCIA02_605_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_257_1/Brats18_TCIA03_257_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_257_1/Brats18_TCIA03_257_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_257_1/Brats18_TCIA03_257_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_257_1/Brats18_TCIA03_257_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_257_1/Brats18_TCIA03_257_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AAP_1/Brats18_CBICA_AAP_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AAP_1/Brats18_CBICA_AAP_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AAP_1/Brats18_CBICA_AAP_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AAP_1/Brats18_CBICA_AAP_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AAP_1/Brats18_CBICA_AAP_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_147_1/Brats18_TCIA01_147_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_147_1/Brats18_TCIA01_147_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_147_1/Brats18_TCIA01_147_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_147_1/Brats18_TCIA01_147_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_147_1/Brats18_TCIA01_147_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_361_1/Brats18_TCIA04_361_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_361_1/Brats18_TCIA04_361_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_361_1/Brats18_TCIA04_361_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_361_1/Brats18_TCIA04_361_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_361_1/Brats18_TCIA04_361_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASN_1/Brats18_CBICA_ASN_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASN_1/Brats18_CBICA_ASN_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASN_1/Brats18_CBICA_ASN_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASN_1/Brats18_CBICA_ASN_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASN_1/Brats18_CBICA_ASN_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_343_1/Brats18_TCIA04_343_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_343_1/Brats18_TCIA04_343_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_343_1/Brats18_TCIA04_343_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_343_1/Brats18_TCIA04_343_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_343_1/Brats18_TCIA04_343_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_455_1/Brats18_TCIA02_455_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_455_1/Brats18_TCIA02_455_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_455_1/Brats18_TCIA02_455_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_455_1/Brats18_TCIA02_455_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_455_1/Brats18_TCIA02_455_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ATB_1/Brats18_CBICA_ATB_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ATB_1/Brats18_CBICA_ATB_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ATB_1/Brats18_CBICA_ATB_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ATB_1/Brats18_CBICA_ATB_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ATB_1/Brats18_CBICA_ATB_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_113_1/Brats18_TCIA08_113_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_113_1/Brats18_TCIA08_113_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_113_1/Brats18_TCIA08_113_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_113_1/Brats18_TCIA08_113_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_113_1/Brats18_TCIA08_113_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_121_1/Brats18_TCIA03_121_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_121_1/Brats18_TCIA03_121_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_121_1/Brats18_TCIA03_121_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_121_1/Brats18_TCIA03_121_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_121_1/Brats18_TCIA03_121_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ABN_1/Brats18_CBICA_ABN_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ABN_1/Brats18_CBICA_ABN_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ABN_1/Brats18_CBICA_ABN_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ABN_1/Brats18_CBICA_ABN_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ABN_1/Brats18_CBICA_ABN_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_162_1/Brats18_TCIA08_162_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_162_1/Brats18_TCIA08_162_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_162_1/Brats18_TCIA08_162_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_162_1/Brats18_TCIA08_162_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_162_1/Brats18_TCIA08_162_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_201_1/Brats18_TCIA01_201_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_201_1/Brats18_TCIA01_201_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_201_1/Brats18_TCIA01_201_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_201_1/Brats18_TCIA01_201_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_201_1/Brats18_TCIA01_201_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_499_1/Brats18_TCIA01_499_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_499_1/Brats18_TCIA01_499_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_499_1/Brats18_TCIA01_499_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_499_1/Brats18_TCIA01_499_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_499_1/Brats18_TCIA01_499_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_4_1/Brats18_2013_4_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_4_1/Brats18_2013_4_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_4_1/Brats18_2013_4_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_4_1/Brats18_2013_4_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_4_1/Brats18_2013_4_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AXQ_1/Brats18_CBICA_AXQ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AXQ_1/Brats18_CBICA_AXQ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AXQ_1/Brats18_CBICA_AXQ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AXQ_1/Brats18_CBICA_AXQ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AXQ_1/Brats18_CBICA_AXQ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AZD_1/Brats18_CBICA_AZD_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AZD_1/Brats18_CBICA_AZD_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AZD_1/Brats18_CBICA_AZD_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AZD_1/Brats18_CBICA_AZD_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AZD_1/Brats18_CBICA_AZD_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_332_1/Brats18_TCIA06_332_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_332_1/Brats18_TCIA06_332_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_332_1/Brats18_TCIA06_332_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_332_1/Brats18_TCIA06_332_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_332_1/Brats18_TCIA06_332_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_18_1/Brats18_2013_18_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_18_1/Brats18_2013_18_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_18_1/Brats18_2013_18_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_18_1/Brats18_2013_18_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_18_1/Brats18_2013_18_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AUQ_1/Brats18_CBICA_AUQ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AUQ_1/Brats18_CBICA_AUQ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AUQ_1/Brats18_CBICA_AUQ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AUQ_1/Brats18_CBICA_AUQ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AUQ_1/Brats18_CBICA_AUQ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_491_1/Brats18_TCIA02_491_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_491_1/Brats18_TCIA02_491_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_491_1/Brats18_TCIA02_491_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_491_1/Brats18_TCIA02_491_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_491_1/Brats18_TCIA02_491_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_165_1/Brats18_TCIA06_165_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_165_1/Brats18_TCIA06_165_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_165_1/Brats18_TCIA06_165_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_165_1/Brats18_TCIA06_165_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_165_1/Brats18_TCIA06_165_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_19_1/Brats18_2013_19_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_19_1/Brats18_2013_19_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_19_1/Brats18_2013_19_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_19_1/Brats18_2013_19_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_19_1/Brats18_2013_19_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_218_1/Brats18_TCIA08_218_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_218_1/Brats18_TCIA08_218_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_218_1/Brats18_TCIA08_218_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_218_1/Brats18_TCIA08_218_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_218_1/Brats18_TCIA08_218_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_150_1/Brats18_TCIA01_150_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_150_1/Brats18_TCIA01_150_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_150_1/Brats18_TCIA01_150_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_150_1/Brats18_TCIA01_150_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_150_1/Brats18_TCIA01_150_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_10_1/Brats18_2013_10_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_10_1/Brats18_2013_10_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_10_1/Brats18_2013_10_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_10_1/Brats18_2013_10_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_10_1/Brats18_2013_10_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_411_1/Brats18_TCIA01_411_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_411_1/Brats18_TCIA01_411_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_411_1/Brats18_TCIA01_411_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_411_1/Brats18_TCIA01_411_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_411_1/Brats18_TCIA01_411_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASE_1/Brats18_CBICA_ASE_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASE_1/Brats18_CBICA_ASE_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASE_1/Brats18_CBICA_ASE_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASE_1/Brats18_CBICA_ASE_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASE_1/Brats18_CBICA_ASE_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_138_1/Brats18_TCIA03_138_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_138_1/Brats18_TCIA03_138_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_138_1/Brats18_TCIA03_138_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_138_1/Brats18_TCIA03_138_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_138_1/Brats18_TCIA03_138_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AWH_1/Brats18_CBICA_AWH_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AWH_1/Brats18_CBICA_AWH_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AWH_1/Brats18_CBICA_AWH_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AWH_1/Brats18_CBICA_AWH_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AWH_1/Brats18_CBICA_AWH_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_171_1/Brats18_TCIA02_171_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_171_1/Brats18_TCIA02_171_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_171_1/Brats18_TCIA02_171_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_171_1/Brats18_TCIA02_171_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_171_1/Brats18_TCIA02_171_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_27_1/Brats18_2013_27_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_27_1/Brats18_2013_27_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_27_1/Brats18_2013_27_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_27_1/Brats18_2013_27_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_27_1/Brats18_2013_27_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_412_1/Brats18_TCIA01_412_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_412_1/Brats18_TCIA01_412_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_412_1/Brats18_TCIA01_412_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_412_1/Brats18_TCIA01_412_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_412_1/Brats18_TCIA01_412_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQN_1/Brats18_CBICA_AQN_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQN_1/Brats18_CBICA_AQN_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQN_1/Brats18_CBICA_AQN_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQN_1/Brats18_CBICA_AQN_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQN_1/Brats18_CBICA_AQN_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQV_1/Brats18_CBICA_AQV_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQV_1/Brats18_CBICA_AQV_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQV_1/Brats18_CBICA_AQV_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQV_1/Brats18_CBICA_AQV_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQV_1/Brats18_CBICA_AQV_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ARZ_1/Brats18_CBICA_ARZ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ARZ_1/Brats18_CBICA_ARZ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ARZ_1/Brats18_CBICA_ARZ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ARZ_1/Brats18_CBICA_ARZ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ARZ_1/Brats18_CBICA_ARZ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_20_1/Brats18_2013_20_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_20_1/Brats18_2013_20_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_20_1/Brats18_2013_20_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_20_1/Brats18_2013_20_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_20_1/Brats18_2013_20_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQR_1/Brats18_CBICA_AQR_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQR_1/Brats18_CBICA_AQR_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQR_1/Brats18_CBICA_AQR_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQR_1/Brats18_CBICA_AQR_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQR_1/Brats18_CBICA_AQR_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_401_1/Brats18_TCIA01_401_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_401_1/Brats18_TCIA01_401_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_401_1/Brats18_TCIA01_401_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_401_1/Brats18_TCIA01_401_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_401_1/Brats18_TCIA01_401_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA05_444_1/Brats18_TCIA05_444_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA05_444_1/Brats18_TCIA05_444_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA05_444_1/Brats18_TCIA05_444_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA05_444_1/Brats18_TCIA05_444_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA05_444_1/Brats18_TCIA05_444_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_338_1/Brats18_TCIA03_338_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_338_1/Brats18_TCIA03_338_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_338_1/Brats18_TCIA03_338_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_338_1/Brats18_TCIA03_338_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_338_1/Brats18_TCIA03_338_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_7_1/Brats18_2013_7_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_7_1/Brats18_2013_7_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_7_1/Brats18_2013_7_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_7_1/Brats18_2013_7_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_7_1/Brats18_2013_7_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AYA_1/Brats18_CBICA_AYA_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AYA_1/Brats18_CBICA_AYA_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AYA_1/Brats18_CBICA_AYA_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AYA_1/Brats18_CBICA_AYA_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AYA_1/Brats18_CBICA_AYA_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQO_1/Brats18_CBICA_AQO_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQO_1/Brats18_CBICA_AQO_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQO_1/Brats18_CBICA_AQO_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQO_1/Brats18_CBICA_AQO_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQO_1/Brats18_CBICA_AQO_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_608_1/Brats18_TCIA02_608_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_608_1/Brats18_TCIA02_608_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_608_1/Brats18_TCIA02_608_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_608_1/Brats18_TCIA02_608_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_608_1/Brats18_TCIA02_608_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AOH_1/Brats18_CBICA_AOH_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AOH_1/Brats18_CBICA_AOH_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AOH_1/Brats18_CBICA_AOH_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AOH_1/Brats18_CBICA_AOH_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AOH_1/Brats18_CBICA_AOH_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_11_1/Brats18_2013_11_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_11_1/Brats18_2013_11_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_11_1/Brats18_2013_11_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_11_1/Brats18_2013_11_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_11_1/Brats18_2013_11_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AXO_1/Brats18_CBICA_AXO_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AXO_1/Brats18_CBICA_AXO_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AXO_1/Brats18_CBICA_AXO_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AXO_1/Brats18_CBICA_AXO_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AXO_1/Brats18_CBICA_AXO_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASG_1/Brats18_CBICA_ASG_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASG_1/Brats18_CBICA_ASG_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASG_1/Brats18_CBICA_ASG_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASG_1/Brats18_CBICA_ASG_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASG_1/Brats18_CBICA_ASG_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_296_1/Brats18_TCIA03_296_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_296_1/Brats18_TCIA03_296_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_296_1/Brats18_TCIA03_296_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_296_1/Brats18_TCIA03_296_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_296_1/Brats18_TCIA03_296_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_14_1/Brats18_2013_14_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_14_1/Brats18_2013_14_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_14_1/Brats18_2013_14_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_14_1/Brats18_2013_14_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_14_1/Brats18_2013_14_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_151_1/Brats18_TCIA02_151_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_151_1/Brats18_TCIA02_151_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_151_1/Brats18_TCIA02_151_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_151_1/Brats18_TCIA02_151_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_151_1/Brats18_TCIA02_151_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_372_1/Brats18_TCIA06_372_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_372_1/Brats18_TCIA06_372_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_372_1/Brats18_TCIA06_372_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_372_1/Brats18_TCIA06_372_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_372_1/Brats18_TCIA06_372_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ALN_1/Brats18_CBICA_ALN_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA05_478_1/Brats18_TCIA05_478_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA05_478_1/Brats18_TCIA05_478_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA05_478_1/Brats18_TCIA05_478_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA05_478_1/Brats18_TCIA05_478_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA05_478_1/Brats18_TCIA05_478_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ANP_1/Brats18_CBICA_ANP_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ANP_1/Brats18_CBICA_ANP_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ANP_1/Brats18_CBICA_ANP_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ANP_1/Brats18_CBICA_ANP_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ANP_1/Brats18_CBICA_ANP_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQQ_1/Brats18_CBICA_AQQ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQQ_1/Brats18_CBICA_AQQ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQQ_1/Brats18_CBICA_AQQ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQQ_1/Brats18_CBICA_AQQ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQQ_1/Brats18_CBICA_AQQ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ARF_1/Brats18_CBICA_ARF_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ARF_1/Brats18_CBICA_ARF_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ARF_1/Brats18_CBICA_ARF_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ARF_1/Brats18_CBICA_ARF_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ARF_1/Brats18_CBICA_ARF_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_469_1/Brats18_TCIA08_469_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_469_1/Brats18_TCIA08_469_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_469_1/Brats18_TCIA08_469_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_469_1/Brats18_TCIA08_469_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_469_1/Brats18_TCIA08_469_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_606_1/Brats18_TCIA02_606_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_606_1/Brats18_TCIA02_606_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_606_1/Brats18_TCIA02_606_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_606_1/Brats18_TCIA02_606_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_606_1/Brats18_TCIA02_606_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_222_1/Brats18_TCIA02_222_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_222_1/Brats18_TCIA02_222_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_222_1/Brats18_TCIA02_222_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_222_1/Brats18_TCIA02_222_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_222_1/Brats18_TCIA02_222_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_105_1/Brats18_TCIA08_105_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_105_1/Brats18_TCIA08_105_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_105_1/Brats18_TCIA08_105_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_105_1/Brats18_TCIA08_105_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_105_1/Brats18_TCIA08_105_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ANI_1/Brats18_CBICA_ANI_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ANI_1/Brats18_CBICA_ANI_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ANI_1/Brats18_CBICA_ANI_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ANI_1/Brats18_CBICA_ANI_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ANI_1/Brats18_CBICA_ANI_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ALX_1/Brats18_CBICA_ALX_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ALX_1/Brats18_CBICA_ALX_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ALX_1/Brats18_CBICA_ALX_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ALX_1/Brats18_CBICA_ALX_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ALX_1/Brats18_CBICA_ALX_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQU_1/Brats18_CBICA_AQU_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQU_1/Brats18_CBICA_AQU_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQU_1/Brats18_CBICA_AQU_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQU_1/Brats18_CBICA_AQU_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQU_1/Brats18_CBICA_AQU_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AWG_1/Brats18_CBICA_AWG_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AWG_1/Brats18_CBICA_AWG_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AWG_1/Brats18_CBICA_AWG_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AWG_1/Brats18_CBICA_AWG_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AWG_1/Brats18_CBICA_AWG_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_BHK_1/Brats18_CBICA_BHK_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_BHK_1/Brats18_CBICA_BHK_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_BHK_1/Brats18_CBICA_BHK_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_BHK_1/Brats18_CBICA_BHK_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_BHK_1/Brats18_CBICA_BHK_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQZ_1/Brats18_CBICA_AQZ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQZ_1/Brats18_CBICA_AQZ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQZ_1/Brats18_CBICA_AQZ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQZ_1/Brats18_CBICA_AQZ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQZ_1/Brats18_CBICA_AQZ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_26_1/Brats18_2013_26_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_26_1/Brats18_2013_26_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_26_1/Brats18_2013_26_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_26_1/Brats18_2013_26_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_26_1/Brats18_2013_26_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_498_1/Brats18_TCIA03_498_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_498_1/Brats18_TCIA03_498_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_498_1/Brats18_TCIA03_498_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_498_1/Brats18_TCIA03_498_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_498_1/Brats18_TCIA03_498_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_17_1/Brats18_2013_17_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_17_1/Brats18_2013_17_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_17_1/Brats18_2013_17_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_17_1/Brats18_2013_17_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_17_1/Brats18_2013_17_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQP_1/Brats18_CBICA_AQP_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQP_1/Brats18_CBICA_AQP_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQP_1/Brats18_CBICA_AQP_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQP_1/Brats18_CBICA_AQP_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQP_1/Brats18_CBICA_AQP_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AVG_1/Brats18_CBICA_AVG_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AVG_1/Brats18_CBICA_AVG_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AVG_1/Brats18_CBICA_AVG_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AVG_1/Brats18_CBICA_AVG_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AVG_1/Brats18_CBICA_AVG_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AYW_1/Brats18_CBICA_AYW_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AYW_1/Brats18_CBICA_AYW_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AYW_1/Brats18_CBICA_AYW_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AYW_1/Brats18_CBICA_AYW_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AYW_1/Brats18_CBICA_AYW_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AOO_1/Brats18_CBICA_AOO_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AOO_1/Brats18_CBICA_AOO_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AOO_1/Brats18_CBICA_AOO_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AOO_1/Brats18_CBICA_AOO_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AOO_1/Brats18_CBICA_AOO_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_322_1/Brats18_TCIA02_322_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_322_1/Brats18_TCIA02_322_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_322_1/Brats18_TCIA02_322_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_322_1/Brats18_TCIA02_322_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_322_1/Brats18_TCIA02_322_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASW_1/Brats18_CBICA_ASW_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASW_1/Brats18_CBICA_ASW_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASW_1/Brats18_CBICA_ASW_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASW_1/Brats18_CBICA_ASW_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASW_1/Brats18_CBICA_ASW_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_226_1/Brats18_TCIA02_226_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_226_1/Brats18_TCIA02_226_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_226_1/Brats18_TCIA02_226_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_226_1/Brats18_TCIA02_226_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_226_1/Brats18_TCIA02_226_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_607_1/Brats18_TCIA02_607_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_607_1/Brats18_TCIA02_607_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_607_1/Brats18_TCIA02_607_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_607_1/Brats18_TCIA02_607_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_607_1/Brats18_TCIA02_607_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AME_1/Brats18_CBICA_AME_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AME_1/Brats18_CBICA_AME_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AME_1/Brats18_CBICA_AME_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AME_1/Brats18_CBICA_AME_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AME_1/Brats18_CBICA_AME_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_603_1/Brats18_TCIA06_603_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_603_1/Brats18_TCIA06_603_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_603_1/Brats18_TCIA06_603_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_603_1/Brats18_TCIA06_603_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_603_1/Brats18_TCIA06_603_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_199_1/Brats18_TCIA03_199_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_199_1/Brats18_TCIA03_199_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_199_1/Brats18_TCIA03_199_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_199_1/Brats18_TCIA03_199_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_199_1/Brats18_TCIA03_199_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AOD_1/Brats18_CBICA_AOD_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AOD_1/Brats18_CBICA_AOD_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AOD_1/Brats18_CBICA_AOD_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AOD_1/Brats18_CBICA_AOD_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AOD_1/Brats18_CBICA_AOD_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AUR_1/Brats18_CBICA_AUR_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AUR_1/Brats18_CBICA_AUR_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AUR_1/Brats18_CBICA_AUR_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AUR_1/Brats18_CBICA_AUR_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AUR_1/Brats18_CBICA_AUR_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_448_1/Brats18_TCIA01_448_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_448_1/Brats18_TCIA01_448_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_448_1/Brats18_TCIA01_448_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_448_1/Brats18_TCIA01_448_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_448_1/Brats18_TCIA01_448_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_368_1/Brats18_TCIA02_368_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_368_1/Brats18_TCIA02_368_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_368_1/Brats18_TCIA02_368_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_368_1/Brats18_TCIA02_368_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_368_1/Brats18_TCIA02_368_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_429_1/Brats18_TCIA01_429_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_429_1/Brats18_TCIA01_429_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_429_1/Brats18_TCIA01_429_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_429_1/Brats18_TCIA01_429_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_429_1/Brats18_TCIA01_429_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AZH_1/Brats18_CBICA_AZH_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AZH_1/Brats18_CBICA_AZH_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AZH_1/Brats18_CBICA_AZH_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AZH_1/Brats18_CBICA_AZH_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AZH_1/Brats18_CBICA_AZH_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_184_1/Brats18_TCIA06_184_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_184_1/Brats18_TCIA06_184_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_184_1/Brats18_TCIA06_184_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_184_1/Brats18_TCIA06_184_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_184_1/Brats18_TCIA06_184_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_23_1/Brats18_2013_23_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_23_1/Brats18_2013_23_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_23_1/Brats18_2013_23_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_23_1/Brats18_2013_23_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_23_1/Brats18_2013_23_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_168_1/Brats18_TCIA02_168_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_168_1/Brats18_TCIA02_168_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_168_1/Brats18_TCIA02_168_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_168_1/Brats18_TCIA02_168_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_168_1/Brats18_TCIA02_168_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_274_1/Brats18_TCIA02_274_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_274_1/Brats18_TCIA02_274_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_274_1/Brats18_TCIA02_274_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_274_1/Brats18_TCIA02_274_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_274_1/Brats18_TCIA02_274_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ATP_1/Brats18_CBICA_ATP_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ATP_1/Brats18_CBICA_ATP_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ATP_1/Brats18_CBICA_ATP_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ATP_1/Brats18_CBICA_ATP_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ATP_1/Brats18_CBICA_ATP_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_300_1/Brats18_TCIA02_300_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_300_1/Brats18_TCIA02_300_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_300_1/Brats18_TCIA02_300_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_300_1/Brats18_TCIA02_300_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_300_1/Brats18_TCIA02_300_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA05_277_1/Brats18_TCIA05_277_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA05_277_1/Brats18_TCIA05_277_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA05_277_1/Brats18_TCIA05_277_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA05_277_1/Brats18_TCIA05_277_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA05_277_1/Brats18_TCIA05_277_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_419_1/Brats18_TCIA03_419_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_419_1/Brats18_TCIA03_419_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_419_1/Brats18_TCIA03_419_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_419_1/Brats18_TCIA03_419_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_419_1/Brats18_TCIA03_419_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_234_1/Brats18_TCIA08_234_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_234_1/Brats18_TCIA08_234_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_234_1/Brats18_TCIA08_234_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_234_1/Brats18_TCIA08_234_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_234_1/Brats18_TCIA08_234_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_335_1/Brats18_TCIA01_335_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_335_1/Brats18_TCIA01_335_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_335_1/Brats18_TCIA01_335_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_335_1/Brats18_TCIA01_335_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_335_1/Brats18_TCIA01_335_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AXJ_1/Brats18_CBICA_AXJ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AXJ_1/Brats18_CBICA_AXJ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AXJ_1/Brats18_CBICA_AXJ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AXJ_1/Brats18_CBICA_AXJ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AXJ_1/Brats18_CBICA_AXJ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_370_1/Brats18_TCIA02_370_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_370_1/Brats18_TCIA02_370_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_370_1/Brats18_TCIA02_370_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_370_1/Brats18_TCIA02_370_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_370_1/Brats18_TCIA02_370_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA03_375_1/Brats18_TCIA03_375_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA03_375_1/Brats18_TCIA03_375_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA03_375_1/Brats18_TCIA03_375_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA03_375_1/Brats18_TCIA03_375_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA03_375_1/Brats18_TCIA03_375_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ANG_1/Brats18_CBICA_ANG_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ANG_1/Brats18_CBICA_ANG_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ANG_1/Brats18_CBICA_ANG_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ANG_1/Brats18_CBICA_ANG_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ANG_1/Brats18_CBICA_ANG_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_211_1/Brats18_TCIA06_211_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_211_1/Brats18_TCIA06_211_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_211_1/Brats18_TCIA06_211_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_211_1/Brats18_TCIA06_211_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_211_1/Brats18_TCIA06_211_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASU_1/Brats18_CBICA_ASU_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASU_1/Brats18_CBICA_ASU_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASU_1/Brats18_CBICA_ASU_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASU_1/Brats18_CBICA_ASU_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASU_1/Brats18_CBICA_ASU_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_283_1/Brats18_TCIA02_283_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_283_1/Brats18_TCIA02_283_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_283_1/Brats18_TCIA02_283_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_283_1/Brats18_TCIA02_283_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_283_1/Brats18_TCIA02_283_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ANZ_1/Brats18_CBICA_ANZ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ANZ_1/Brats18_CBICA_ANZ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ANZ_1/Brats18_CBICA_ANZ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ANZ_1/Brats18_CBICA_ANZ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ANZ_1/Brats18_CBICA_ANZ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ABY_1/Brats18_CBICA_ABY_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ABY_1/Brats18_CBICA_ABY_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ABY_1/Brats18_CBICA_ABY_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ABY_1/Brats18_CBICA_ABY_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ABY_1/Brats18_CBICA_ABY_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_328_1/Brats18_TCIA04_328_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_328_1/Brats18_TCIA04_328_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_328_1/Brats18_TCIA04_328_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_328_1/Brats18_TCIA04_328_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_328_1/Brats18_TCIA04_328_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_118_1/Brats18_TCIA02_118_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_118_1/Brats18_TCIA02_118_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_118_1/Brats18_TCIA02_118_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_118_1/Brats18_TCIA02_118_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_118_1/Brats18_TCIA02_118_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_BFB_1/Brats18_CBICA_BFB_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_BFB_1/Brats18_CBICA_BFB_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_BFB_1/Brats18_CBICA_BFB_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_BFB_1/Brats18_CBICA_BFB_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_BFB_1/Brats18_CBICA_BFB_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_2013_5_1/Brats18_2013_5_1_t1.nii.gz', 't2': 'HGG/Brats18_2013_5_1/Brats18_2013_5_1_t2.nii.gz', 't1ce': 'HGG/Brats18_2013_5_1/Brats18_2013_5_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_2013_5_1/Brats18_2013_5_1_flair.nii.gz', 'seg': 'HGG/Brats18_2013_5_1/Brats18_2013_5_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AVJ_1/Brats18_CBICA_AVJ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AVJ_1/Brats18_CBICA_AVJ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AVJ_1/Brats18_CBICA_AVJ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AVJ_1/Brats18_CBICA_AVJ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AVJ_1/Brats18_CBICA_AVJ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AAG_1/Brats18_CBICA_AAG_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AAG_1/Brats18_CBICA_AAG_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AAG_1/Brats18_CBICA_AAG_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AAG_1/Brats18_CBICA_AAG_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AAG_1/Brats18_CBICA_AAG_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQD_1/Brats18_CBICA_AQD_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQD_1/Brats18_CBICA_AQD_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQD_1/Brats18_CBICA_AQD_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQD_1/Brats18_CBICA_AQD_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQD_1/Brats18_CBICA_AQD_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASH_1/Brats18_CBICA_ASH_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASH_1/Brats18_CBICA_ASH_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASH_1/Brats18_CBICA_ASH_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASH_1/Brats18_CBICA_ASH_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASH_1/Brats18_CBICA_ASH_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_APY_1/Brats18_CBICA_APY_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_APY_1/Brats18_CBICA_APY_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_APY_1/Brats18_CBICA_APY_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_APY_1/Brats18_CBICA_APY_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_APY_1/Brats18_CBICA_APY_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AUN_1/Brats18_CBICA_AUN_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AUN_1/Brats18_CBICA_AUN_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AUN_1/Brats18_CBICA_AUN_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AUN_1/Brats18_CBICA_AUN_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AUN_1/Brats18_CBICA_AUN_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_167_1/Brats18_TCIA08_167_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_167_1/Brats18_TCIA08_167_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_167_1/Brats18_TCIA08_167_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_167_1/Brats18_TCIA08_167_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_167_1/Brats18_TCIA08_167_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQJ_1/Brats18_CBICA_AQJ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQJ_1/Brats18_CBICA_AQJ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQJ_1/Brats18_CBICA_AQJ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQJ_1/Brats18_CBICA_AQJ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQJ_1/Brats18_CBICA_AQJ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA04_149_1/Brats18_TCIA04_149_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA04_149_1/Brats18_TCIA04_149_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA04_149_1/Brats18_TCIA04_149_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA04_149_1/Brats18_TCIA04_149_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA04_149_1/Brats18_TCIA04_149_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA06_247_1/Brats18_TCIA06_247_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA06_247_1/Brats18_TCIA06_247_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA06_247_1/Brats18_TCIA06_247_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA06_247_1/Brats18_TCIA06_247_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA06_247_1/Brats18_TCIA06_247_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_APZ_1/Brats18_CBICA_APZ_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_APZ_1/Brats18_CBICA_APZ_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_APZ_1/Brats18_CBICA_APZ_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_APZ_1/Brats18_CBICA_APZ_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_APZ_1/Brats18_CBICA_APZ_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_378_1/Brats18_TCIA01_378_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_378_1/Brats18_TCIA01_378_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_378_1/Brats18_TCIA01_378_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_378_1/Brats18_TCIA01_378_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_378_1/Brats18_TCIA01_378_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_321_1/Brats18_TCIA02_321_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_321_1/Brats18_TCIA02_321_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_321_1/Brats18_TCIA02_321_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_321_1/Brats18_TCIA02_321_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_321_1/Brats18_TCIA02_321_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_278_1/Brats18_TCIA08_278_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_278_1/Brats18_TCIA08_278_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_278_1/Brats18_TCIA08_278_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_278_1/Brats18_TCIA08_278_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_278_1/Brats18_TCIA08_278_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQA_1/Brats18_CBICA_AQA_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQA_1/Brats18_CBICA_AQA_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQA_1/Brats18_CBICA_AQA_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQA_1/Brats18_CBICA_AQA_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQA_1/Brats18_CBICA_AQA_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA01_231_1/Brats18_TCIA01_231_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA01_231_1/Brats18_TCIA01_231_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA01_231_1/Brats18_TCIA01_231_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA01_231_1/Brats18_TCIA01_231_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA01_231_1/Brats18_TCIA01_231_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ATF_1/Brats18_CBICA_ATF_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ATF_1/Brats18_CBICA_ATF_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ATF_1/Brats18_CBICA_ATF_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ATF_1/Brats18_CBICA_ATF_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ATF_1/Brats18_CBICA_ATF_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ABB_1/Brats18_CBICA_ABB_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ABB_1/Brats18_CBICA_ABB_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ABB_1/Brats18_CBICA_ABB_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ABB_1/Brats18_CBICA_ABB_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ABB_1/Brats18_CBICA_ABB_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ALU_1/Brats18_CBICA_ALU_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ALU_1/Brats18_CBICA_ALU_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ALU_1/Brats18_CBICA_ALU_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ALU_1/Brats18_CBICA_ALU_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ALU_1/Brats18_CBICA_ALU_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AXW_1/Brats18_CBICA_AXW_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AXW_1/Brats18_CBICA_AXW_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AXW_1/Brats18_CBICA_AXW_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AXW_1/Brats18_CBICA_AXW_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AXW_1/Brats18_CBICA_AXW_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA08_242_1/Brats18_TCIA08_242_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA08_242_1/Brats18_TCIA08_242_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA08_242_1/Brats18_TCIA08_242_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA08_242_1/Brats18_TCIA08_242_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA08_242_1/Brats18_TCIA08_242_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_473_1/Brats18_TCIA02_473_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_473_1/Brats18_TCIA02_473_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_473_1/Brats18_TCIA02_473_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_473_1/Brats18_TCIA02_473_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_473_1/Brats18_TCIA02_473_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AAB_1/Brats18_CBICA_AAB_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_AQY_1/Brats18_CBICA_AQY_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_AQY_1/Brats18_CBICA_AQY_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_AQY_1/Brats18_CBICA_AQY_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_AQY_1/Brats18_CBICA_AQY_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_AQY_1/Brats18_CBICA_AQY_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_TCIA02_198_1/Brats18_TCIA02_198_1_t1.nii.gz', 't2': 'HGG/Brats18_TCIA02_198_1/Brats18_TCIA02_198_1_t2.nii.gz', 't1ce': 'HGG/Brats18_TCIA02_198_1/Brats18_TCIA02_198_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_TCIA02_198_1/Brats18_TCIA02_198_1_flair.nii.gz', 'seg': 'HGG/Brats18_TCIA02_198_1/Brats18_TCIA02_198_1_seg.nii.gz'}, {'t1': 'HGG/Brats18_CBICA_ASA_1/Brats18_CBICA_ASA_1_t1.nii.gz', 't2': 'HGG/Brats18_CBICA_ASA_1/Brats18_CBICA_ASA_1_t2.nii.gz', 't1ce': 'HGG/Brats18_CBICA_ASA_1/Brats18_CBICA_ASA_1_t1ce.nii.gz', 'flair': 'HGG/Brats18_CBICA_ASA_1/Brats18_CBICA_ASA_1_flair.nii.gz', 'seg': 'HGG/Brats18_CBICA_ASA_1/Brats18_CBICA_ASA_1_seg.nii.gz'}]











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
kernel = np.ones((3, 3))
File = []
Binary = []
Gray = []
Processed_Binary = []
Processed_Gray = []

data_x = np.empty((1,) + input_shape, dtype=np.float32)
labels = np.empty((1, output_channels) + input_shape[1:], dtype=np.uint8)

for index,i in enumerate(samples[177:]):
    File.append(i['t1'].split('/')[1])
    print("Sr no. %s/%s File: %s"%(index+1, len(samples[177:]),File[-1]))  
    
    data_x[0] = np.array([preprocess(read_img(i[m]), input_shape[1:]) for m in ['t1', 't2', 't1ce', 'flair']], dtype=np.float32)
    labels[0] = preprocess_label(read_img(i['seg']), input_shape[1:])[None, ...]
    
    preds = model.predict(data_x)[0]
    
    pbinary = [] #predicted binary
    pgray = [] #predicted grayscale
    label_gray = [] #original gray
    label_binary = [] #original binary

    binary = [] #processed binary
    gray = [] #processed grayscale
    original_gray = [] #processed original gray
    original_binary = [] #processed original binary
    for i in range(96):
        pred = preds[:, i,:,:]
        pred = np.transpose(pred, (1, 2, 0))
        pred = np.rint(pred)
        pred = pred*255
        pred_gray = cv2.cvtColor(pred, cv2.COLOR_BGR2GRAY)
        pred_gray = pred_gray.astype(np.uint8)
        pgray.append(pred_gray)
        pbinary.append(np.array(pred_gray>0).astype(int))
        pred_gray_closed = cv2.morphologyEx(pred_gray, cv2.MORPH_CLOSE, kernel, iterations=3)
        gray.append(pred_gray_closed)
        pred_binary = np.array(pred_gray_closed>0).astype(int)
        binary.append(pred_binary)
        orig = labels[0][:, i,:,:]
        orig = np.transpose(orig, (1, 2, 0))
        orig = orig*255
        orig_gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
        orig_gray = orig_gray.astype(np.uint8)
        label_gray.append(orig_gray)
        label_binary.append(np.array(orig_gray>0).astype(int))
        orig_gray_closed = cv2.morphologyEx(orig_gray, cv2.MORPH_CLOSE, kernel, iterations=3)
        original_gray.append(orig_gray_closed)
        orig_binary = np.array(orig_gray_closed>0).astype(int)
        original_binary.append(orig_binary)
        
    Binary.append(K.get_value(dice_coefficient(K.constant(np.array(label_binary)), K.constant(np.array(pbinary)))))
    Gray.append(K.get_value(dice_coefficient(K.constant(np.array(label_gray)), K.constant(np.array(pgray)))))
    Processed_Binary.append(K.get_value(dice_coefficient(K.constant(np.array(original_binary)), K.constant(np.array(binary)))))
    Processed_Gray.append(K.get_value(dice_coefficient(K.constant(np.array(original_gray)), K.constant(np.array(gray)))))
    print("Dice score without post processing for binary: %s"%Binary[-1])
    print("Dice score without post processing for gray: %s"%Gray[-1])
    print("Dice score with post processing binary: %s"%Processed_Binary[-1])
    print("Dice score with post processing gray: %s\n"%Processed_Gray[-1])

Sr no. 1/98 File: Brats18_TCIA05_444_1


RuntimeError: ignored

In [0]:
# # samples[177:]
# print(len(File))
# # print(len(Binary))
# # print(len(Gray))
# # print(len(Processed_Binary))
# # print(len(Processed_Gray))
# # File.pop()

177


In [0]:
df = pd.DataFrame({
    'File': File,
    'Binary': Binary,
    'Gray': Gray,
    'Processed_Binary': Processed_Binary,
    'Processed_Gray': Processed_Gray
})
df.to_csv('train_accuracy_177_above.csv', index=False)